<a href="https://colab.research.google.com/github/Shibu4064/ML_Project/blob/main/buetcsenlpbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#from google.colab import drive
import pandas as pd

# # Mount Google Drive
# drive.mount('/content/drive')

# Load CSV Data
csv_file_path = '/content/train(5).csv'
df = pd.read_csv(csv_file_path)

# Define parameters
text_column = 'text'
text_length = 300

# Filter Rows
filtered_df = df[df[text_column].str.len() != text_length]

# Save Result
filtered_csv_path = '/content/train_last.csv'
filtered_df.to_csv(filtered_csv_path, index=False)


In [3]:
df1=pd.read_csv('/content/train_last.csv')
df1

,text,label
0,এখানে আরো ভালো ভাবে দলীয় ও র এর অবস্থান পাকা হ...,1
1,চুয়াডাঙ্গা বাড়ি কে বলেছে আপনার,1
2,"ভাই সোনাই ঘোষ এর দই খেয়ে যাইতেন , খুব ই মজার",2
3,সমার তালুকদার আপনার ছবিতে ফেসটা কেন জানি বন্য ...,0
4,ভাইয়া এই নুডলস টা কোথায় কিনতে পাওয়া যাবে প্লিজ...,2
...,...,...
35257,সরকারি খাদ্যগুদাম থেকে নষ্ট চাল জব্দ করার ঘটনা...,2
35258,যুগোপযোগী শিক্ষা গ্রহণ না করাই এর মূল কারণ ।,0
35259,রূপগঞ্জে অগ্নিকাণ্ড: ঢামেক মর্গে সারি সারি মরদ...,0
35260,সমঝোতায় আসার সিদ্ধান্ত নিয়েছেন জাপানি নারী ম...,2


In [4]:
text_column = 'text'
text_length = 1000

# Filter Rows
filtered_df = df1[df1[text_column].str.len() < text_length]
filtered_df.to_csv('train_csv',index=False)

In [5]:
train_texts=filtered_df['text']
for idx, text in enumerate(train_texts):
  if len(text)>=1000:print("Found 1000")

In [6]:
df2=pd.read_csv('/content/train_csv')
df2

,text,label
0,এখানে আরো ভালো ভাবে দলীয় ও র এর অবস্থান পাকা হ...,1
1,চুয়াডাঙ্গা বাড়ি কে বলেছে আপনার,1
2,"ভাই সোনাই ঘোষ এর দই খেয়ে যাইতেন , খুব ই মজার",2
3,সমার তালুকদার আপনার ছবিতে ফেসটা কেন জানি বন্য ...,0
4,ভাইয়া এই নুডলস টা কোথায় কিনতে পাওয়া যাবে প্লিজ...,2
...,...,...
35185,সরকারি খাদ্যগুদাম থেকে নষ্ট চাল জব্দ করার ঘটনা...,2
35186,যুগোপযোগী শিক্ষা গ্রহণ না করাই এর মূল কারণ ।,0
35187,রূপগঞ্জে অগ্নিকাণ্ড: ঢামেক মর্গে সারি সারি মরদ...,0
35188,সমঝোতায় আসার সিদ্ধান্ত নিয়েছেন জাপানি নারী ম...,2


In [7]:
%%capture
!pip install transformers datasets
!pip install transformers
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]
!pip install torch
!pip install tensorflow
!pip install evaluate

In [8]:
train_df=pd.read_csv('/content/train_csv')
val_df=pd.read_csv('/content/val(2).csv')

In [9]:
print(len(train_df))

35190


In [10]:
train_df.columns

Index(['text', 'label'], dtype='object')

In [11]:
from sklearn.model_selection import train_test_split
def pandaToList(dataset):
    text_list = []
    label_list = []
    for i in range(len(dataset['text'])):
      text_list.append(dataset['text'][i])
      label_list.append(int(dataset['label'][i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
val_texts, val_labels = pandaToList(val_df)

In [12]:
tokenizer_name = "csebuetnlp/banglabert"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)

In [13]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
import torch

class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels)
val_dataset = ViolenceDataset(val_encodings, val_labels)


In [15]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [16]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertTokenizer, BertModel
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   #output_dir= output_dir,
   eval_steps = 250, # Evaluation and Save happens every 250 steps
   save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.

   learning_rate=1e-5,
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   num_train_epochs=50,
   weight_decay=0.01,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

model_name = 'csebuetnlp/banglabert'
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3) ## change here for using another pretrained model

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
trainer.train()

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,No log,1.026385,0.486274,0.327348,0.396560,0.353734
500,1.060100,0.936431,0.603711,0.422027,0.491510,0.441788
750,1.060100,0.896715,0.626843,0.418509,0.518714,0.462424
1000,1.039400,1.254983,0.631418,0.437042,0.516286,0.464163
1250,1.039400,1.436020,0.631927,0.422125,0.531168,0.468123
1500,1.322500,1.744523,0.624301,0.444298,0.506962,0.456524
1750,1.322500,1.753839,0.565074,0.451156,0.447556,0.393102
2000,1.517000,1.456582,0.646670,0.429728,0.540046,0.478323
2250,1.517000,1.584122,0.644382,0.428655,0.539085,0.476726
2500,1.318500,1.596145,0.654550,0.434848,0.545091,0.483756


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=9250, training_loss=1.2759858728357263, metrics={'train_runtime': 5839.4456, 'train_samples_per_second': 301.313, 'train_steps_per_second': 301.313, 'total_flos': 1269188209831500.0, 'train_loss': 1.2759858728357263, 'epoch': 0.26})

In [18]:
trainer.predict(val_dataset)

PredictionOutput(predictions=array([[ 3.8383086 , -1.6117947 , -2.4187686 ],
       [-2.116799  , -1.5585139 ,  4.2319074 ],
       [ 3.579473  , -1.5647516 , -2.1767502 ],
       ...,
       [-2.0654685 , -1.5062155 ,  4.1122317 ],
       [-0.10238626,  0.94747555, -1.035282  ],
       [ 3.7175963 , -1.6152066 , -2.290344  ]], dtype=float32), label_ids=array([0, 2, 0, ..., 2, 2, 0]), metrics={'test_loss': 1.4085510969161987, 'test_accuracy': 0.6753940010167768, 'test_precision': 0.6274436118527745, 'test_recall': 0.6130717822538047, 'test_f1': 0.6152663380129219, 'test_runtime': 115.6057, 'test_samples_per_second': 34.029, 'test_steps_per_second': 34.029})

In [19]:
output_dir = "/content/drive/MyDrive/VITD-Models/buetcsenlpbert"

In [20]:
trainer.save_model(f"{output_dir}") # saving the model to drive.